# Exploratory parsing

Goals:
- Figure out how to extract a structured version of the change request to a law.
- Figure out how to parse a law in a structured format that allows for modification by change requests


Data source: 
- Änderungsgesetz: https://dip.bundestag.de/experten-suche?term=NOT%20zusatzmerkmal:E%20AND%20vorgangstyp_notation:100&f.wahlperiode=19&f.typ=Vorgang&rows=25&sort=datum_ab
- Source law: https://www.gesetze-im-internet.de/wregg/BJNR273910017.html

Use anytree python package for tree datastructure: https://anytree.readthedocs.io/en/latest/

### Todo

- [ ] handle multiple changes in one change line

## Read data

In [1]:
import pdfplumber
import regex as re

In [2]:
# open pdf
filename = "../data/0483-21.pdf"


def read_pdf_law(filename: str) -> str:
    """Get the raw text from the pdfs."""
    # read all pages from provided pdf
    pdf_file_obj = pdfplumber.open(filename)

    # join the pages
    return "".join(
        [page for page in [page.extract_text() for page in pdf_file_obj.pages] if page]
    ).replace("-\n", "")


change_law_raw = read_pdf_law(filename)

## Structure and clean up the text

In [3]:
def preprocess_raw_law(text: str) -> str:
    """Apply some preprocessing to the raw text of the laws.

    Every line in the output starts with a "bullet point identifier" (e.g. § 2, (1), b), aa))

    Args:
        text: string containing the law text.

    Returns:
        String with preprocessing applied.
    """
    # extract the parts with change requests (here we assume only one law is affected for now)
    # > get the text between "wird wie folgt geändert" und "Begründung" (allow for newlines and/or whitespace between the words)
    text = re.split(
        r"wird[\s,\n]{0,3}wie[\s,\n]{0,3}folgt[\s,\n]{0,3}geändert:", text, maxsplit=1
    )[1].split("Begründung", 1)[0]

    # remove newlines between quotation marks
    # > text in quotation marks is text that should be replaced or modified in the affected law.
    # > Since there can be §, sections or other bulletpoint identifiers,
    # > we remove all newlines in this text (so lines don't start with bullet point identifiers)
    for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", text, re.MULTILINE):
        text = (
            text[: m.span()[0]]
            + text[m.span()[0] : m.span()[1]].replace("\n", " ")
            + text[m.span()[1] :]
        )

    # remove text artifacts from the page
    text = re.sub(r"\.?Drucksache \d{2,3}\/\d{1,2}", "", text)  # Drucksache...
    text = re.sub(r"- \d -", "", text)  # page numbering
    text = text.strip()  # remove trailing whitespace or newlines

    # pull every bulletpoint content to one line
    outtext = ""
    for line_num, line in enumerate(text.split("\n")):
        # check if line starts with a bullet point identifier
        # > if yes, put it in a new line, otherwise just append the linetext to the text
        if any(
            [
                re.match(r"^\d\.", line),
                re.match(r"^[a-z]\)", line),
                re.match(r"^[a-z][a-z]\)", line),
                re.match(r"^\([a-z1-9]\)", line),
            ]
        ):
            outtext += "\n" + line
        else:
            outtext += line

    return outtext.strip()


clean_change_law = preprocess_raw_law(change_law_raw)

In [4]:
def expand_text(text: str) -> str:
    """Expand the lines, such that every line is one complete change request.

    Args:
        text: preprocessed text.

    Returns:
        Expanded text. Every line contains one change request text.
    """
    outtext = ""
    stack = []
    for line in text.split("\n"):
        if re.match(r"^\d\.", line):
            if stack:
                outtext += " ".join(stack) + "\n"
            stack = []
            stack.append(line)
        if re.match(r"^[a-z]\)", line):
            if len(stack) >= 2:
                outtext += " ".join(stack) + "\n"
                stack.pop()
            stack.append(line)
        if re.match(r"^[a-z][a-z]\)", line):
            if len(stack) >= 3:
                outtext += " ".join(stack) + "\n"
                stack.pop()
            stack.append(line)
    if stack:
        outtext += " ".join(stack) + "\n"
    return outtext.strip()


clean_text = expand_text(clean_change_law)

In [5]:
print(clean_text)

1.  § 2 wird wie folgt geändert:  a)  Nach Absatz 2 wird folgender neuer Absatz 3 eingefügt: „(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von  vergleichbarer  Schwere  wie  die  Straf-  und  Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere  Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob  fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1  und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen  gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen  Arbeitsorganisation.  Der für die Eintragung erforderliche Nachweis der jeweiligen schweren  Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein  vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung  verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige  Behörde zu treffen.“
1.  § 2 wird wie folgt g

## Parse changes

In [6]:
from typing import List, Tuple


def parse_change_location(line: str) -> List[str]:
    """Parse the location identifiers from one line of change request.

    Find the location identifiers (i.e. Absatz, § 4, Satz 5, etc) in the text and return these.

    Args:
        line: One line of text. Contains exactly one change request.

    Returns:
        List of strings, every string is one step of the location to change.
    """
    location_identifiers = [
        r"Inhaltsübersicht?",
        r"§ \d{1,3}[a-z]?",
        r"Überschrift",
        r"Absatz \d{1,3}",
        r"Satz \d{1,3}",
        r"Nummer \d{1,3}",
        r"Buchstabe [a-z]",
    ]

    location = []
    for loc_ident in location_identifiers:
        try:
            # search location identifier in text
            location.extend(
                re.search(
                    loc_ident,
                    re.sub(
                        r"(?<=„)(.|\n)*?(?=“)", "", line
                    ),  # don't search in quoted text
                ).captures()
            )
        except:
            # if the identifier is not found, pass
            pass
    return location


def parse_change_text(line: str) -> List[str]:
    """Parse the text that needs to be changed from one line of change request.

    Look for text in quotation marks and return it.

    Args:
        line: One line of text. Contains exactly one change request.

    Returns:
        List of strings, every string is one change text.
    """
    return [
        line[m.span()[0] : m.span()[1]].replace("Komma", ",").replace("Semikolon", ";")
        for m in re.finditer(
            r"((?<=„)(.|\n)*?(?=“)|Komma|Semikolon)", line, re.MULTILINE
        )
    ]


def parse_change_request_line(line: str) -> List[dict]:
    """Parse the actions of one line of change requests.

    Look for certain keywords in the line (i.e. "eingefügt", "gestrichen", etc) to identify what should be done in this change.
    Then parse change location and change text.

    Args:
        line: One line of text. Contains exactly one change request.

    Returns:
        A list of dicts with required changes extracted from this line.
    """
    res = []
    if "eingefügt" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "insert_after",
        }
        res.append(res_dict)
    elif "ersetzt" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "replace",
        }
        res.append(res_dict)
    elif "gefasst" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "rephrase",
        }
        res.append(res_dict)
    elif "angefügt" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "append",
        }
        res.append(res_dict)
    elif "gestrichen" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "delete_after",
        }
        res.append(res_dict)
    elif "aufgehoben" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "cancelled",
        }
        res.append(res_dict)
    else:
        # We assume every line is a change, so if nothing is found, we don't know yet how to handle it.
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "UNKNOWN",
        }
        res.append(res_dict)

    return res

Lets look at the change requests.

In [7]:
changes = []
for change_request_line in clean_text.split("\n"):
    print(change_request_line)
    res = parse_change_request_line(change_request_line)
    print()
    print(res)
    changes.extend(res)
    print()
    print(150 * "#")
    print()

1.  § 2 wird wie folgt geändert:  a)  Nach Absatz 2 wird folgender neuer Absatz 3 eingefügt: „(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von  vergleichbarer  Schwere  wie  die  Straf-  und  Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere  Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob  fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1  und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen  gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen  Arbeitsorganisation.  Der für die Eintragung erforderliche Nachweis der jeweiligen schweren  Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein  vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung  verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige  Behörde zu treffen.“

[{'location': ['§ 2', '

## Parse the source law from raw text

Here we try to parse a source law into a structured form.

In [8]:
source_text = open("../data/0483-21_wettbewerbsgesetzt.txt", "r").read()

Use [anytree](https://anytree.readthedocs.io/en/latest/) to build a tree for that law.

In [9]:
from anytree import NodeMixin, RenderTree, findall


class LawTextNode(NodeMixin):
    def __init__(self, text, bulletpoint, parent=None, children=None):
        self.text = text
        self.bulletpoint = bulletpoint
        self.parent = parent
        if children:  # set children only if given
            self.children = children

    def __repr__(self):
        return "{} - {}".format(self.bulletpoint, self.text)

    def _to_text(self):
        """return the full law text"""
        treestr = ""
        for pre, fill, node in RenderTree(self):
            treestr += "{}{} {}\n".format(" " * len(pre), node.bulletpoint, node.text)
        return treestr

    def _print(self):
        for pre, fill, node in RenderTree(self):
            treestr = u"{}{} - {}".format(pre, node.bulletpoint, node.text)
            print(treestr.ljust(8))

In [29]:
def parse_source_law_tree(
    text: str, source_node=LawTextNode(text="source", bulletpoint="source")
) -> LawTextNode:
    """Parse raw law text into a structured format.

    Look for bullet point patters and build a tree with it.

    Args:
        text: raw text of the law.

    Returns:
        Structured output. A tree of LawTextNodes.
    """
    patterns = [
        r"\n§\s\d{1,3}[a-z]?",
        r"\n\s*\([a-z1-9]\)",
        r"\n\s*\d{1,2}\.",
        r"\n\s*[a-z]\)",
    ]

    # build the tree
    for pattern in patterns:
        used_texts = []
        # search the pattern in the text
        if re.search(pattern, text):
            split_text = re.split(pattern, text)
            for idx, m in enumerate(re.finditer(pattern, text)):
                new_node = LawTextNode(
                    text=split_text[idx + 1].strip().split("\n")[0],
                    # store the text for this bullet point on this level
                    bulletpoint=text[m.span()[0] : m.span()[1]].strip(),
                    # apply the function recursively to get all levels
                    parent=source_node,
                )
                # get the next level associated with this node
                _ = parse_source_law_tree(split_text[idx + 1], source_node=new_node)
                used_texts.append(split_text[idx + 1])
        for ut in used_texts:
            text = text.replace(ut, "")
    return source_node


parsed_law_tree = parse_source_law_tree(source_text)

In [30]:
parsed_law_tree._print()

source - source
├── § 1 - Einrichtung des Wettbewerbsregisters
│   ├── (1) - Beim Bundeskartellamt (Registerbehörde) wird ein Register zum Schutz des Wettbewerbs um öffentliche Aufträge und Konzessionen (Wettbewerbsregister) eingerichtet und geführt.
│   ├── (2) - Mit dem Wettbewerbsregister werden Auftraggebern im Sinne von § 98 des Gesetzes gegen Wettbewerbsbeschränkungen Informationen über Ausschlussgründe im Sinne der §§ 123 und 124 des Gesetzes gegen Wettbewerbsbeschränkungen zur Verfügung gestellt.
│   └── (3) - Das Wettbewerbsregister wird in Form einer elektronischen Datenbank geführt.
├── § 2 - Eintragungsvoraussetzungen
│   ├── (1) - In das Wettbewerbsregister sind einzutragen:
│   │   ├── 1. - rechtskräftige strafgerichtliche Verurteilungen und Strafbefehle, die wegen einer der folgenden Straftaten ergangen sind:
│   │   │   ├── a) - in § 123 Absatz 1 des Gesetzes gegen Wettbewerbsbeschränkungen aufgeführte Straftaten,
│   │   │   ├── b) - Betrug nach § 263 des Strafgesetzbu

## Apply the changes to the source law

Here we try to figure out, how to apply the required changes to the parsed source law.

In [12]:
# find path to node


def _find_path(location_list: List[str], parse_tree: LawTextNode) -> List[LawTextNode]:
    path = []
    res = parse_tree
    for loc in location_list:
        if loc.startswith("Absatz "):
            loc = loc.replace("Absatz ", "(") + ")"
        elif loc.startswith("Nummer "):
            loc = loc.replace("Nummer ", "") + "."
        elif loc.startswith("Buchstabe "):
            loc = loc.replace("Buchstabe ", "") + ")"
        try:
            res = findall(res, filter_=lambda node: loc == node.bulletpoint)[0]
            path.append(res)
        except:
            # print("Location {} not found".format(loc))
            pass
    return path

In [13]:
import copy


def _replace(
    law_tree: LawTextNode, path: List[LawTextNode], change: dict
) -> LawTextNode:
    # apply replace operation to the last text in the path
    if len(change["text"]) == 2:
        path[-1].text = path[-1].text.replace(change["text"][0], change["text"][1])
    else:
        print("not enougth text to replace")
    return law_tree


def _insert_after(
    law_tree: LawTextNode, path: List[LawTextNode], change: dict
) -> LawTextNode:
    # insert text after a given text
    # assumes list of change["text"] to be of even length, uneven positions are the location after which to insert, even positions are the text to insert.
    if len(change["text"]) % 2 == 0:
        for idx in range(len(change["text"]) // 2):
            path[-1].text = re.sub(
                r"\b{}\b".format(change["text"][2 * idx]),
                change["text"][2 * idx] + " " + change["text"][2 * idx + 1],
                path[-1].text,
            )
    elif len(change["text"]) == 1 and any(
        [
            re.match(r"^\d\.", change["text"][0]),
            re.match(r"^[a-z]\)", change["text"][0]),
            re.match(r"^[a-z][a-z]\)", change["text"][0]),
            re.match(r"^\([a-z1-9]\)", change["text"][0]),
        ]
    ):
        # if there is only one text to insert and it starts with a bulletidentifier, add a new node to the tree
        _ = LawTextNode(
            text=change["text"][0], bulletpoint="(new)", parent=path[-1].parent
        )
    else:
        print("Failed to insert after! Not enought texts.")
    return law_tree


def _rephrase(
    law_tree: LawTextNode, path: List[LawTextNode], change: dict
) -> LawTextNode:
    return law_tree


def _delete_after(
    law_tree: LawTextNode, path: List[LawTextNode], change: dict
) -> LawTextNode:
    if len(change["text"]) == 1:
        path[-1].text = path[-1].text.replace(change["text"][0], "")
    elif len(change["text"]) > 1:
        path[-1].text = path[-1].text.replace("".join(change["text"][1:]), "")
    else:
        print("Failed to delete! Not enought text.")
    return law_tree


def apply_changes(law_tree: LawTextNode, changes: dict) -> LawTextNode:
    res_law_tree = copy.deepcopy(law_tree)
    for change in changes:
        path = _find_path(location_list=change["location"], parse_tree=res_law_tree)
        change_type = change["how"]
        if change_type == "replace":
            print("APPLIED {}:\n{}".format(change_type, change))
            res_law_tree = _replace(res_law_tree, path, change)
        elif change_type == "insert_after":
            print("APPLIED {}:\n{}".format(change_type, change))
            res_law_tree = _insert_after(res_law_tree, path, change)
        # elif change_type == "rephrase":
        #    print("APPLIED {}:\n{}".format(change_type, change))
        #    res_law_tree = _rephrase(res_law_tree, path, change)
        elif change_type == "delete_after":
            print("APPLIED {}:\n{}".format(change_type, change))
            res_law_tree = _delete_after(res_law_tree, path, change)
        else:
            print("SKIPPED {}:\n{}".format(change_type, change))
    return res_law_tree

In [14]:
res_law_tree = apply_changes(parsed_law_tree, changes)

APPLIED insert_after:
{'location': ['§ 2', 'Absatz 2'], 'text': ['(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von  vergleichbarer  Schwere  wie  die  Straf-  und  Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere  Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob  fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1  und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen  gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen  Arbeitsorganisation.  Der für die Eintragung erforderliche Nachweis der jeweiligen schweren  Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein  vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung  verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige  Behörde zu treffen.'], 'how': 'insert_after'}
APPLIED insert_after:
{'loc

In [15]:
res_law_tree._print()

source - source
├── § 1 - Einrichtung des Wettbewerbsregisters
│   ├── (1) - Beim Bundeskartellamt (Registerbehörde) wird ein Register zum Schutz des Wettbewerbs um öffentliche Aufträge und Konzessionen (Wettbewerbsregister) eingerichtet und geführt.
│   ├── (2) - Mit dem Wettbewerbsregister werden Auftraggebern im Sinne von § 98 des Gesetzes gegen Wettbewerbsbeschränkungen Informationen über Ausschlussgründe im Sinne der §§ 123 und 124 des Gesetzes gegen Wettbewerbsbeschränkungen zur Verfügung gestellt.
│   └── (3) - Das Wettbewerbsregister wird in Form einer elektronischen Datenbank geführt.
├── § 2 - Eintragungsvoraussetzungen
│   ├── (1) - In das Wettbewerbsregister sind einzutragen:
│   │   ├── 1. - rechtskräftige strafgerichtliche Verurteilungen und Strafbefehle, die wegen einer der folgenden Straftaten ergangen sind:
│   │   │   ├── a) - in § 123 Absatz 1 des Gesetzes gegen Wettbewerbsbeschränkungen aufgeführte Straftaten,
│   │   │   ├── b) - Betrug nach § 263 des Strafgesetzbu

### Get the law text back

In [16]:
print(res_law_tree._to_text())

source source
    § 1 Einrichtung des Wettbewerbsregisters
        (1) Beim Bundeskartellamt (Registerbehörde) wird ein Register zum Schutz des Wettbewerbs um öffentliche Aufträge und Konzessionen (Wettbewerbsregister) eingerichtet und geführt.
        (2) Mit dem Wettbewerbsregister werden Auftraggebern im Sinne von § 98 des Gesetzes gegen Wettbewerbsbeschränkungen Informationen über Ausschlussgründe im Sinne der §§ 123 und 124 des Gesetzes gegen Wettbewerbsbeschränkungen zur Verfügung gestellt.
        (3) Das Wettbewerbsregister wird in Form einer elektronischen Datenbank geführt.
    § 2 Eintragungsvoraussetzungen
        (1) In das Wettbewerbsregister sind einzutragen:
            1. rechtskräftige strafgerichtliche Verurteilungen und Strafbefehle, die wegen einer der folgenden Straftaten ergangen sind:
                a) in § 123 Absatz 1 des Gesetzes gegen Wettbewerbsbeschränkungen aufgeführte Straftaten,
                b) Betrug nach § 263 des Strafgesetzbuchs und Subventionsb

## Apply it all to another document

In [17]:
# try a second document
filename2 = "../data/0145-21.pdf"

change_law_raw2 = read_pdf_law(filename2)

In [18]:
clean_change_law2 = preprocess_raw_law(change_law_raw2)
clean_text2 = expand_text(clean_change_law2)

for t in clean_text2.split("\n")[:15]:
    print(t)

1. Die Inhaltsübersicht wird wie folgt geändert: a) Der Angabe zu § 130a werden ein Semikolon und das Wort „Verordnungsermächtigung“ angefügt.
1. Die Inhaltsübersicht wird wie folgt geändert: b) Die Angaben zu den §§ 173 bis 176 werden wie folgt gefasst:„§ 173 Zustellung elektronischer Dokumente §174 Zustellung durch Aushändigung an der Amtsstelle §175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis §176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag“.
2. § 130a wird wie folgt geändert: a) Der Überschrift werden ein Semikolon und das Wort „Verordnungsermächtigung“angefügt.
2. § 130a wird wie folgt geändert: b) Absatz 2 Satz 2 wird wie folgt gefasst:„Die Bundesregierung bestimmt durch Rechtsverordnung mit Zustimmung des Bundesrates technische Rahmenbedingungen für die Übermittlung und Eignung zur Bearbeitung durch das Gericht.“
2. § 130a wird wie folgt geändert: c) Absatz 4 wird wie folgt geändert: aa) In Nummer 3 werden nach dem Wort „Gerichts“ das Semikolon u

In [19]:
changes = []
for change_request_line in clean_text2.split("\n"):
    res = parse_change_request_line(change_request_line)
    print(change_request_line)
    print()
    print(res)
    changes.extend(res)
    print()
    print(50 * "#")
    print()

1. Die Inhaltsübersicht wird wie folgt geändert: a) Der Angabe zu § 130a werden ein Semikolon und das Wort „Verordnungsermächtigung“ angefügt.

[{'location': ['Inhaltsübersicht', '§ 130a'], 'text': [';', 'Verordnungsermächtigung'], 'how': 'append'}]

##################################################

1. Die Inhaltsübersicht wird wie folgt geändert: b) Die Angaben zu den §§ 173 bis 176 werden wie folgt gefasst:„§ 173 Zustellung elektronischer Dokumente §174 Zustellung durch Aushändigung an der Amtsstelle §175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis §176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag“.

[{'location': ['Inhaltsübersicht', '§ 173'], 'text': ['§ 173 Zustellung elektronischer Dokumente §174 Zustellung durch Aushändigung an der Amtsstelle §175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis §176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag'], 'how': 'rephrase'}]

##################################################
